Use Sparkmagic (PySpark) kernel for this notebook
Get csv from internet source and clean it then save to S3 bucket
The csv can be read by another nootbook Covid-19-Pandas.ipynb(with conda_python3 kernel) for visualization

In [ ]:
import boto3
import pandas as pd
from io import StringIO # python3; python2: BytesIO 

data_url ='https://bit.ly/3d93pa1'

pdf = pd.read_csv(data_url)

Transpose and extract selected columns(countries) 

In [ ]:
pdf1 = pdf.T.drop(['Country/Region','Province/State','Lat','Long']).iloc[:,[28,116,120,137,201,225]]

Add a 'date' column to df2 and update column names

In [ ]:
pdf1['date'] = pd.date_range(start='1/22/2020', periods=len(pdf1), freq='D')
pdf1.columns = ['Brazil', 'France', 'Germany', 'Italy', 'Spain', 'US', 'date']

In [ ]:
print(pdf1)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# a scatter plot comparing num_children and num_pets
pdf1.plot('date',['US', 'Brazil', 'Italy', 'France', 'Germany'])
%matplot plt


Save pdf1 to S3 as csv

In [ ]:
bucket = 'mybucket' # already created on S3
csv_buffer = StringIO()
pdf1.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'pandas/cv.csv').put(Body=csv_buffer.getvalue())

Convert Pandas dataframe (pdf1) to Spark dataframe (sdf)

In [ ]:
from pyspark.sql import *
sdf = spark.createDataFrame(pdf1)

In [ ]:
sdf.show(10)

In [ ]:
sdf.printSchema()

Save spark dataframe to S3 with parquet format, this way the schema is preserved, csv doesn't 
preserve schema, so it will become messy when read

In [ ]:
sdf.write.parquet("s3a://mybucket/parquet/",mode="overwrite")

Now read it back to another spark dataframe sdf1

In [ ]:
sdf1 = spark.read.load("s3n://mybucket/parquet/")
sdf1.show(5)

Save sdf to hive table

In [ ]:
sdf.createOrReplaceTempView("mytempTable") 

In [ ]:
sqlContext.sql("drop table if exists CV19")
sqlContext.sql("create table if not exists CV19 as select * from mytempTable");

In [ ]:
sqlContext.sql("select * from CV19").show()

Or simply just use %%sql to run the queries

In [ ]:
%%sql
select * from CV19 where US >= 3000000

Read from csv in S3 to Pandas dataframe

In [ ]:
import boto3
import pandas as pd
import io

In [ ]:
s3c = boto3.client('s3')
obj = s3c.get_object(Bucket= 'mybucket' , Key = "pandas/cv.csv")
pdf3 = pd.read_csv(io.BytesIO(obj['Body'].read()), encoding='utf8')

Now plot out a pie chart using the last row confirmed cases

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Brazil', 'France', 'Germany', 'Italy', 'Spain', 'US'
confirmed = pdf3.tail(1).iloc[0,1:7]
explode = (0, 0, 0, 0, 0, 0.1)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
ax1.pie(confirmed, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

In [ ]:
%matplot plt

Print out the current date when the data is collected

In [ ]:
print(pdf3.tail(1).iloc[0,7])